In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
import io
from sklearn.metrics import precision_recall_fscore_support
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix

Train = pd.read_csv("Train.csv")
stop_words = stopwords.words("english")
f = io.open("words.txt", mode="r", encoding="utf-8")
stop_words = set(stopwords.words('english')) 
wordlist = f.read().split("\n")
wordlist = [word for word in wordlist if word not in stop_words]
f.close()

In [2]:
Train.Review = Train['Review'].map(lambda a: re.sub(r'[^\w\s]','',a).lower())
Train.Review = Train['Review'].map(lambda a: ' '.join([word for word in a.split() if word not in stop_words]))

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(vocabulary = wordlist, encoding="ISO-8859-1", strip_accents=ascii, analyzer='word')
bagofwords_train = vectorizer.fit_transform(Train['Review'])

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
tf_idf_train = tfidf_transformer.fit_transform(bagofwords_train)
NBModel = MultinomialNB().fit(tf_idf_train, Train['Class'])

In [5]:
Test = pd.read_csv("imdb_ts.csv")
y_true = Test['Class']
Test.Review = Test['Review'].map(lambda a: re.sub(r'[^\w\s]','',a).lower())
Test.Review = Test['Review'].map(lambda a: ' '.join([word for word in a.split() if word not in stop_words]))
vectorizer = CountVectorizer(vocabulary = wordlist, encoding="ISO-8859-1", strip_accents=ascii, analyzer='word')
bagofwords_test = vectorizer.fit_transform(Test['Review'])
tfidf_transformer = TfidfTransformer()
tf_idf_test = tfidf_transformer.fit_transform(bagofwords_test)

In [10]:
y_pred = NBModel.predict(tf_idf_test)
print(confusion_matrix(y_true, y_pred))
print(accuracy_score(y_true, y_pred))
print(precision_score(y_true, y_pred))
print(recall_score(y_true, y_pred))

[[10979  1521]
 [ 2599  9901]]
0.8352
0.8668359306601295
0.79208
